# bqplot

Welcome to bqplot! This notebook will serve as an introduction to bqplot and the widgets framework of the Jupyter Notebook. 

### So, what is bqplot?

`bqplot` is a Grammar of Graphics based interactive visualization library for the Jupyter notebook.

Uhh, so - what does that mean?

In [132]:
## Let's see!

In [133]:
# Let's begin by importing some libraries we'll need
import numpy as np
import pandas as pd
import os
from __future__ import print_function # So that this notebook becomes both Python 2 and Python 3 compatible

# And creating some random data
size = 100
np.random.seed(0)
x_data = np.arange(size)
y_data = np.cumsum(np.random.randn(size)  * 100.0)

In [134]:
# We'll start with bqplot's matplotlib inspired API

from bqplot import pyplot as plt

It means that any plot you generate is, by default, an interactive web element that you can control using **only** Python.

In [135]:
figure = plt.figure(title='My First Plot')
scatter = plt.scatter(x_data, y_data)
plt.show()

A Jupyter Widget

It also means that any aspect of the plot **is an interactive widget**. This allows us to change any aspect of the plot - after it's been drawn.

#### Try it! Run the cell below a few times and watch the plot above change!

In [81]:
scatter.y = np.cumsum(np.random.randn(size)  * 100.0)

Since a bqplot chart is a web element, we should be able to animate it right?

### Let's animate it!

In [76]:
figure.animation_duration = 500

Voila!

In [82]:
scatter.y = np.cumsum(np.random.randn(size)  * 100.0)

That doesn't apply to just the x and y. To have a look at the attributes, take a look at the documentation!

In [83]:
# Say, the color
scatter.colors = ['Red']

In [84]:
# Or, the marker style
scatter.marker = 'diamond'

It's important to remember that an interactive widget means that the `JavaScript` and the `Python` communicate. So, the plot can be changed through a single line of python code, or a piece of python code can be triggered by a change in the plot. Let's go through a simple example. Say we have a function `foo`:

In [88]:
def foo(change):
    #print('Hello World')
    print(change)

We can call `foo` everytime any attribute of our scatter is changed. Say, the `y` values:

In [89]:
# First, we hook up our function `foo` to the colors attribute (or Trait) of the scatter plot
scatter.observe(foo, 'y')

To allow the points in the `Scatter` to be moved interactively, we set the `enable_move` attribute to `True`

In [90]:
scatter.enable_move = True

Go ahead, head over to the chart and move any point in some way. This move (which happens on the `JavaScript` side should trigger our `Python` function `foo`.

## That's great, but what does that have to do with the Grammar of Graphics?

`bqplot` has two different APIs. One is the matplotlib inspired `pyplot` which we used above (you can think of it as similar to `qplot` in `ggplot2`). The other one, the verbose API, is meant to expose every element of a plot individually, so that their attriutes can be controlled in an atomic way. In order to truly use `bqplot` to build complex and feature-rich GUIs, it pays to understand the underlying theory that is used to create a plot.

**A `Scale` is a mapping from (function that converts) data coordinates to figure coordinates.** What this means is that, a `Scale` takes a set of values in any arbitrary unit (say number of people, or $, or litres) and converts it to pixels (or colors for a `ColorScale`).

In [91]:
# First, we import the scales
from bqplot import LinearScale

In [92]:
# Let's create a scale for the x attribute, and a scale for the y attribute
x_sc = LinearScale()
y_sc = LinearScale()

Now, we need to create the actual `Mark` that will visually represent the data. Let's pick a `Scatter` chart to start.

In [93]:
from bqplot import Scatter

scatter_chart = Scatter(x=x_data[:20], y=y_data[:20], scales={'x': x_sc, 'y': y_sc})

Most of the time, the actual `Figure` co-ordinates don't really mean anything to us. So, what we need is the visual representation of our `Scale`, which is called an `Axis`.

In [94]:
from bqplot import Axis

x_ax = Axis(label='X', scale=x_sc)
y_ax = Axis(label='Y', scale=y_sc, orientation='vertical')

And finally, we put it all together on a canvas, which is called a `Figure`.

In [95]:
from bqplot import Figure

fig = Figure(marks=[scatter_chart], title='A Figure', axes=[x_ax, y_ax])
fig

A Jupyter Widget

Now, that the plot has been generated, we can control every single attribute of it. Let's say we wanted to color the chart based on some other data.

In [96]:
# First, we generate some random color data.
color_data = np.random.randint(0, 2, size=20)

In [97]:
color_data

array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0])

Now, we define a ColorScale to map the color_data to actual colors

In [98]:
from bqplot import ColorScale

# The colors trait controls the actual colors we want to map to. It can also take a min, mid, max list of
# colors to be interpolated between for continuous data.
col_sc = ColorScale(colors=['MediumSeaGreen', 'Red'])

In [99]:
scatter_chart.scales = {'x': x_sc, 'y': y_sc, 'color': col_sc}
# We pass the color data to the Scatter Chart through it's color attribute
scatter_chart.color = color_data

The grammar of graphics framework allows us to overlay multiple visualizations on a single `Figure` by having the visualization share the `Scales`. So, for example, if we had a `Bar` chart that we would like to plot alongside the `Scatter` plot, we just pass it the same `Scales`.

In [100]:
from bqplot import Bars

scale = 100.
x_data_new = np.arange(20)
y_data_new = np.cumsum(np.random.randn(20)  * scale)

In [101]:
# All we need to do to add a bar chart to the Figure is pass the same scales to the Mark
bar_chart = Bars(x=x_data_new, y=y_data_new, scales={'x': x_sc, 'y': y_sc}, colors=['deepskyblue'])

Finally, we add the new `Mark` to the `Figure` to update the plot!

In [102]:
fig.marks = [bar_chart, scatter_chart]

## Overview of bqplot `Marks` 

Checkout the [bqplot examples](https://github.com/bloomberg/bqplot/tree/master/examples/Marks) for a simple walkthrough of all the basic visualizations that bqplot offers

## Interactions

### Tooltips

Tooltips can be made to display an attribute of the data:

In [103]:
county_data = pd.read_csv(os.path.abspath('data_files/2008-election-results.csv'))
winner = np.array(['McCain'] * county_data.shape[0])
winner[(county_data['Obama'] > county_data['McCain']).values] = 'Obama'

In [104]:
from bqplot import Tooltip, AlbersUSA, OrdinalColorScale, ColorAxis

In [105]:
sc_geo_county = AlbersUSA()
sc_c1_county = OrdinalColorScale(domain=['McCain', 'Obama'], colors=['Red', 'DeepSkyBlue'])
color_data_county = dict(zip(county_data['FIPS'].values.astype(int), list(winner)))

map_styles_county = {'color': color_data_county,
              'scales': {'projection': sc_geo_county, 'color': sc_c1_county}, 'colors': {'default_color': 'Grey'}}

county_fig = plt.figure(title='US Elections 2008 - Example')
county_map = plt.geo(map_data='USCountiesMap', **map_styles_county)

In [106]:
map_tt = Tooltip(fields=['id', 'name', 'color'], labels=['County Code', 'County Name', 'Winner'])

In [107]:
county_map.tooltip = map_tt

In [108]:
county_fig

A Jupyter Widget

Alternately, any instance of a widget can be made a `Tooltip`

In [109]:
tt_fig = plt.figure(title='It works!')
tt_line = plt.plot(x_data, y_data)

In [110]:
map_fig = plt.figure(title='World Map')
world_map = plt.geo(map_data='WorldMap', tooltip=tt_fig)
map_fig

A Jupyter Widget

**Exercise**: Use the `on_hover` function provided by the `Map` class to generate a specific tooltip for each country.

## Selections

In [136]:
fig = plt.figure(title='My First Plot')
scatter_plot = plt.scatter(x_data, y_data, interactions={'click': 'select'})

fig

A Jupyter Widget

In [137]:
scatter_plot.selected

[]

In [144]:
scatter_plot.unselected_style={'opacity': 0.4}
scatter_plot.selected_style={'fill': 'red', 'stroke': 'yellow', 'width': '1125px', 'height': '125px'}

`selected` is a trait. So we can always observe it.

In [115]:
def observe_selected(change):
    print('The selected indexes are {}'.format(scatter_plot.selected))

scatter_plot.observe(observe_selected, 'selected')

`selected` can also be set using `Selectors`.

In [116]:
def call_back(name, value):
    print(str(value))
    
_ = plt.brush_selector(call_back)

## Mixing bqplot with ipywidgets

The entire framework for bqplot is based on `d3js` (on the JavaScript side) and on the [ipywidgets](www.github.com/ipywidgets) framework. This means that integrating it with native Jupyter widgets is a seamless experience. As is integrating it with other widget based libraries, such as:

- ipyleaflet
- ipyvolume
- pythreejs

In [117]:
from sklearn.linear_model import LinearRegression

In [118]:
## Let's generate some noisy data

x_lin = np.linspace(0, 1, 15)
y_lin = 1.1 * x_lin + .3 * np.random.randn(x_lin.shape[0])

In [119]:
## Fitting a linear regression to this model

lin_reg = LinearRegression()
_ = lin_reg.fit(x_lin[:, np.newaxis], y_lin)

In [120]:
scat_fig = plt.figure(title='Linear Regression')
scat = plt.scatter(x_lin, y_lin, enable_move=True, restrict_y=True)
reg_line = plt.plot(x_lin, lin_reg.predict(x_lin[:, np.newaxis]), colors=['Red'])

## Everytime a point is moved, we recalibrate the model

In [121]:
def point_moved(change):
    lin_reg = LinearRegression()
    lin_reg.fit(x_lin[:, np.newaxis], scat.y)
    reg_line.y = lin_reg.predict(x_lin[:, np.newaxis])
    
scat.observe(point_moved, 'y')

In [122]:
scat_fig

A Jupyter Widget

### Let's tie the line to a slider

In [123]:
from ipywidgets import IntSlider, VBox

In [124]:
x_test = np.linspace(-1., 1., 100)

def update_regression(x_train, y_train, x_test, order):
    y_test = np.zeros_like(x_test)
    y_test = np.polyval(np.polyfit(x_train, y_train, order), x_test)
    return y_test
    
def params_changed(change):
    y_vals = update_regression(scat.x, scat.y, x_test, order_slider.value)
    reg_line.y = y_vals

In [125]:
x_train = np.linspace(-0.8, 0.8, 10)
y_train = 1.1 * (x_train ** 2) + .3 * np.random.randn(x_train.shape[0])

In [126]:
## slider to decide the order of the regression
order_slider = IntSlider(description='Order', min=1, max=5, value=1)
order_slider.observe(params_changed, 'value')

scat_fig = plt.figure(title='Polynomial Regression', animation_duration=100)

## click on the figure to add points to the scatter.
## this shows how our model responds to new data.
scat = plt.scatter(x_train, y_train, interactions={'click': 'add'})
reg_line = plt.plot(x_test, update_regression(x_train, y_train, x_test, order_slider.value),
                    colors=['Red'])
scat.observe(params_changed, ['x', 'y'])

In [127]:
VBox([order_slider, scat_fig])

A Jupyter Widget